In [1]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)


In [2]:
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)


In [3]:
for X, y in data_iter:
    print(X, y)
    break


tensor([[-0.3851,  0.6994],
        [-1.3341, -0.5789],
        [ 1.1998, -1.4357],
        [-0.1888, -0.8898],
        [-0.0539, -0.0402],
        [-0.1100, -0.0733],
        [-0.2712, -2.1745],
        [ 2.1550,  0.3655],
        [-2.1865, -1.0002],
        [ 1.2573,  0.4709]]) tensor([ 1.0601,  3.5057, 11.4761,  6.8557,  4.2389,  4.2287, 11.0527,  7.2657,
         3.2229,  5.0990])


In [5]:

import torch.nn as nn

class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构


LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [6]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])


Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [7]:
for param in net.parameters():
    print(param)


Parameter containing:
tensor([[-0.0142,  0.1194]], requires_grad=True)
Parameter containing:
tensor([0.0535], requires_grad=True)


In [8]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)


Parameter containing:
tensor([0.], requires_grad=True)

In [9]:
loss = nn.MSELoss()


In [10]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)


SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [ ]:
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)


In [ ]:
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍


In [11]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))


epoch 1, loss: 0.000231
epoch 2, loss: 0.000102
epoch 3, loss: 0.000039


In [12]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)


[2, -3.4] Parameter containing:
tensor([[ 1.9995, -3.4004]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1995], requires_grad=True)


使用PyTorch可以更简洁地实现模型。
torch.utils.data模块提供了有关数据处理的工具，torch.nn模块定义了大量神经网络的层，torch.nn.init模块定义了各种初始化方法，torch.optim模块提供了很多常用的优化算法。